In [ ]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, BucketIterator
import numpy as np
import pandas as pd
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
!pip install inltk
from inltk.inltk import tokenize
from torchtext import data
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

!pip install Morfessor

# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

! pip install --user indic
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

from indicnlp import loader
loader.load()

from indicnlp.tokenize import indic_tokenize 
import re
from indicnlp.tokenize import indic_detokenize  
from indicnlp.transliterate.unicode_transliterate import ItransTransliterator
import spacy

nlp = spacy.load("en_core_web_sm")


In [ ]:
# HINDI AND ENGLISH TOKENIZERS  :::::


tok_hindi= []
tok_english = []

co=1

def tokenize_hindi(text):
  #global co
  #co =co+1
  #print("fd  " , co)
  #indic_string='लिए @ तरहQuiver♫एकज्या नाड़ीके♪ - [सिहरन]'
  indic_string = text

  res=""
#print('Input String: {}'.format(indic_string))
#print('Tokens: ')
  for text in indic_tokenize.trivial_tokenize(indic_string): 
   
    #print(text)
    text=text.replace('„', r'')
    text=text.replace('“', r'')
    text=text.replace('”', r'')
    text=text.replace('–', r'')
    text=text.replace('—', r'')
    text=text.replace('´', r"")
    text=text.replace('‘', r"")
    text=text.replace('‚', r"")
    text=text.replace('’', r"")
    text=text.replace("''", r'')
    text=text.replace('´´', r'')
    text=text.replace('…', r'')
    text=text.replace('(',r'')
    text=text.replace(')',r'')
    text=text.replace('.',r'')
    text=text.replace(',',r'')
    text=text.replace('/',r'')
    text=text.replace('[',r'')
    text=text.replace(']',r'')
    text=text.replace('♫',r'')
    text=text.replace('♪',r'')
    text=text.replace('-',r'')
    text=text.replace('.',r'')
    text=text.replace('।',r'')
    text=text.replace('|',r'')
    res=res+' '+text

  reaesc = re.compile(r'[a-zA-Z]')
  res = reaesc.sub('',res)
  reaesc = re.compile(r'[@_♫♪!#$%^&*()?/|}{~:.]')
  res = reaesc.sub('',res)
   
  return indic_tokenize.trivial_tokenize(res)



def tokenize_english(l2):
  
  l1 = []
  #print("ec")
  reaesc = re.compile(r'[@_!♫♪#$%^&*(.,)<>?/\|}{~:]')
  text = reaesc.sub('',l2)
  #print(text)
  text=text.replace("-","")
  text=text.replace("fuckin'","fucking")
  text=text.replace("'ii"," will")
  text=text.replace("'ll", r" will")
  text=text.replace("can't", r"can not")
  text=text.replace("won't", r"will not")
  text=text.replace("n't", r" not")
  text=text.replace("'re", r" are")
  text=text.replace("i'm", r"i am")
  text=text.replace("'s" , r" is")
  text=text.replace("'ve", r" have")
  text=text.replace("'ve" , r"have")
  text=text.replace("cannot" , r"can not")

  m1 = ""
  

  doc = nlp(text)
 # for token in indic_tokenize.trivial_tokenize(l2):
  
  for token in doc:
    reg = re.compile(r'[A-Za-z0-9]')
    #print(token.text)
    if reg.match(token.text):
      l1.append(token.text)
  
  length = len(l1)

  list1 = []
  #print(l1)
  for i in range(0,length-1):
   # print(i,l1[i] , length)
    if l1[i]=='ai' and l1[i+1]=='not':
      l1[i] = "ain't"
      i=i+1
      list1.append(i)
  
  
  #print(list1)
  list2 = []

  for i in range(0,len(l1)):
     if i not in list1:
       list2.append(l1[i])
  return list2
  
  #return indic_tokenize.trivial_tokenize(l2)

  #return (tokenize(res,'en'))

In [ ]:
df = pd.read_csv("train.csv")
#print(df)
size = df.shape[0]
l1 = 90000
l2 = 10000
l3 = size - l1 -l2

#Removed the indexing from the csv file
df = df.drop(df.columns[0], axis=1)

# In this initially during training time , I divided the data into training and validation set , but in order to test
# over the dev test trained it over all the present inputs 
df[0:size].to_csv(f"tra_data.csv", sep =',', index=False, index_label=False)
df[100000:100020].to_csv(f"val_data.csv", sep =',', index=False, index_label=False)

In [ ]:
# Firstly , fields are constructed , that will contain the information to process the data . It will take as parameter , the init_token (to be appended to
# to the starting of the sentence) , the eos_token (to be appended to the end of the sentence) , (lower=True to convert the words to lowercase)
# and the appropriate tokenizer to tokenize the input 
hindi = Field(sequential=True, use_vocab=True, init_token="<sos>", eos_token="<eos>" , lower=True, tokenize=tokenize_hindi)

english = Field(sequential=True, use_vocab=True, init_token="<sos>", eos_token="<eos>" , lower=True,  tokenize=tokenize_english)


# The fields to be constructed for the following columns - hindi and english
fields = [
  ('hindi', hindi),
  ('english', english)
]

# loading the dataset into train_data(training data) and valid_data(validation data)
train_data, valid_data = data.TabularDataset.splits(
   path = '/content',
   train = 'tra_data.csv',
   validation = 'val_data.csv',
   format = 'csv',
   fields = fields,
   skip_header = True)


In [ ]:
#Construct the Vocab object for this field from datasets 
# I Have taken the vocab size to be 50000 after experimenting with number of different values
# Build vocab -> define a vocabulary object with attributes as frequency(store the frequency of tokens) , stoi(string to index mapping)
# ans itos(index to string mapping)
hindi.build_vocab(train_data, max_size=50000)
english.build_vocab(train_data, max_size=500000)


In [ ]:
batch_size=32
# Creates batches of data such that the sentences which are of similar lengths are batched together
# in the same batch and they make the size of the sentence in each batch equal by adding padding
# at the end of the sentences which are of lesser length than the largest available sentence
train_iterator, valid_iterator= BucketIterator.splits((train_data, valid_data), 
                                                                      batch_size = batch_size, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.hindi),
                                                                      device = device)


In [ ]:
class Encode(nn.Module):
  def __init__(self, e_input, e_embedding,
                           hidden_size, num_layers, dropout):
    super(Encode, self).__init__()

    # Number of layers in the lstm
    self.num_layers = num_layers


    # number of features for LSTM to remember
    self.hidden_size = hidden_size

    # No of final nodes to drop in order to avoid overfitting
    self.dropout = nn.Dropout(dropout)
    self.tag = True

    # applying a embedding on the given input_size. The input size here is len(hindi.vocab.stoi) which basically
    # tells us that we need to make an embedding corresponding to every word present in our model so that at the 
    # time of testing , we can get the embedding corresponding to any word we want . The embedding basically represents each corresponding 
    # word into multiple dimensions.  

    # We can use -->print(self.embedding.weight) to get the required embedding -> will also show requires_gradient = True which shows
    # it is learnable i.e. it will learn the embeddings for the words that will be passed to it during the forward pass.
    #     print((self.embedding.weight).size()) => will give input_size X embedding_size tensor

    self.embedding = nn.Embedding(e_input, e_embedding)
    
    #Applies a multi-layer long short-term memory (LSTM) RNN to an input sequence.
    self.LSTM = nn.LSTM(e_embedding, hidden_size, num_layers,dropout = dropout)

    # Displays the weights and biases , choose the bias->0 as tried putting 1/4 to 1/2 of bias length=1
    # in order to LSTM to remember more by default but that did not improve the results on the validation set.
    #m = self.LSTM._all_weights
    #print(self.LSTM.bias_ih_l0.data[0:0])
    #self.LSTM.bias_ih_l0.data[20:40]=1
    #self.LSTM.bias_ih_l1.data[20:40]=1
    #self.LSTM.bias_hh_l0.data[20:40]=1
    #self.LSTM.bias_hh_l1.data[20:40]=1

  # Dimensions of x [Sentence_length, batch_size](Every sentence of the batch will have the same size since padding was used to make them of equal size)
  def forward(self, x):

    
    # Dimensions of embedding[Sequence_length , batch_size , embedding dims]
    # Here , the words of the sentence x for all the sentences of the batch size (32)
    # will be learning an embedding which was initially created using the constructor and function above
    # i.e. nn.Embedding 
    embedding1 = self.embedding(x)
                
    # Avoid overfitting by dropping some of the nodes
    embedding = self.dropout(embedding1)
    
    #Dimensions of output [Sequence_length , batch_size , hidden_size]
    #Dimensions of hidden state and cell state [num_layers, batch_size size, hidden_size]
    # The embedding is then passed through the LSTM layer which uses the hidden state and cell state + previously obtained outputs
    encoder_out , (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state


In [ ]:
class Decode(nn.Module):
  def __init__(self, d_input, d_embedding,hidden_size, num_layers, drop, result_size):
    
    super(Decode, self).__init__()


    # Number of layers in the lstm
    self.num_layers = num_layers
    # Output size of the word embedding NN
    self.embedding_size = d_embedding
    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size
    # The result which is obtained after the LSTM model results are passed through
    # a fully connected layer to give the final vector which contains a probability
    # corresponding to each word in the vocabulary.
    self.result_size = result_size

    # Regularization parameter
    self.dropout = nn.Dropout(drop)

    # Shape (hindi_vocab_size,embedding_size) [input size, embedding dims]
    #The module that allows you to use Embedding is torch.nn.Embedding.
    #It takes two parameters : the vocabulary size and the dimensionality of the embedding
    self.embedding = nn.Embedding(d_input, d_embedding)

    # Shape(embedding_size,no_of_layers_of_lstm,hidden_size) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(d_embedding, hidden_size, num_layers, dropout = drop)

    # Tried putting 1/4 to 1/2 of bias length=1
    # in order to LSTM to remember more by default but that did not improve the results on the validation set.
    # Similar to the earlier case of encoders
    #m = self.e_LSTM._all_weights
    #print(self.e_LSTM.bias_ih_l0.data[0:0])
    #self.e_LSTM.bias_ih_l0.data[20:40]=1
    #self.e_LSTM.bias_ih_l1.data[20:40]=1
    #self.e_LSTM.bias_hh_l0.data[20:40]=1
    #self.e_LSTM.bias_hh_l1.data[20:40]=1


    # Shape(hidden_size,english_vocab_size) [embedding dims, hidden size, num layers]
    #For fully connected layer, number of input features = number of hidden units in LSTM. 
    self.fc = nn.Linear(hidden_size, result_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape(1, 32, embedding_Size) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape of outputs (1, 32, hidden_size) [1, batch_size , hidden_size]
    # Shape(no_of_layers_in_lstm, 32 ,hidden_size)[num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    results, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

     
    # Shape of predictions (1, 32, 4556) [ 1, batch_size , output_size]
    #Before putting the lstm output into fc layer it has to be flattened out as done above using
    # nn.Linear
     #Output Size =  1*32*english_vocab_size -> as for each word in all the 64 batches simulatenously
    # processing , there would be a prob associated with every word present in the vocab and the word with 
    # max prob is choosen as the value to be fed to the next LSTM input to 1st layer (this is done later on
    # in the training step)
    english_pred = self.fc(results)

    # Shape --> predictions [batch_size , output_size]
    english_pred = english_pred.squeeze(0)

    return english_pred, hidden_state, cell_state



In [ ]:
batch_size=32
input_size_encoder = len(hindi.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5
encoder_lstm = Encode(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)
input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = Decode(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)
# THE SIZE OF THE INPUT TO THE DECODER and size of the result size would be same as the input to
# the decoder is one word from each of the batch that is fed to the system of lstm simultaneously and the output
# would be 1 word for all the 32 batches.
learning_rate = 0.001

# SGD Did not perform good for both the encoders and the decoders and Adam gave the best possible performance.
#e_opt = optim.SGD(encoder_lstm.parameters(), lr=learning_rate)
e_opt = optim.Adam(encoder_lstm.parameters(), lr=learning_rate)
#d_opt = optim.SGD(decoder_lstm.parameters(), lr=learning_rate)
d_opt = optim.Adam(decoder_lstm.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=1)

Encode(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(41993, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)
Decode(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(32814, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=32814, bias=True)
)


In [ ]:
def get_best_words(input_sentence,largest_sentence,device):
  with torch.no_grad():
    hidden, cell = encoder_lstm.forward(input_sentence)
    
    outputs = [2]
   # We append a <sos> token to the start of the string and then get the output from the decoder lstm and then 
   # get the outputs in the output till we don't encounter a <eos> token.
   # At the end we will get the index of the best prediction set of words.
    for p in range(largest_sentence):
      last_output_word = torch.LongTensor([outputs[-1]]).to(device)

      with torch.no_grad():
        output, hidden, cell = decoder_lstm.forward(last_output_word, hidden, cell)
        bg1,bg2 = torch.max(output, 1)
        highest_prob_word = bg2.item()
        
      outputs.append(highest_prob_word)

      if highest_prob_word == 3:
        break
  return outputs

In [ ]:
def predictions(sentence, hindi, english, device, largest_sentence=500):
     
    reaesc = re.compile(r'[a-zA-Z]')
    res = reaesc.sub(' ',sentence)
    reaesc = re.compile(r'[@_!#$%^&*()<>?/|}{~:]')
    res = reaesc.sub(' ',res)
    hind_list = indic_tokenize.trivial_tokenize(res)
    hind_list.insert(0, hindi.init_token)
    hind_list.append(hindi.eos_token)
    # Tokenized the hindi sentence obtained from the dev set .. appended sos and eos token at the start and end of the token list
    # and then converted the set of words in the tokens to the corresponding indices and hence we get the sentence to be fed as input
    # to the encoder ...which on receiving the input , using the LSTMS return the hidden and cell state , which when fed to the decoder ,gives the
    # required output from which the maximum probability word is choosen.
    # At the end the final english sentence is generated and returned.
    token_index=[]

    for word in hind_list:
	    token_index.append(hindi.vocab.stoi[word])
    
    input_sentence = torch.tensor(token_index).unsqueeze(1).to(device)
   
    outputs = get_best_words(input_sentence,largest_sentence,device)
  
    translated_sentence=[]
    # Translate the obtain indices to the required words
    for val in outputs:
		    translated_sentence.append(english.vocab.itos[val])
    
    if("<eos>" in translated_sentence):
      translated_sentence.remove("<eos>")
    if("<sos>" in translated_sentence):
      translated_sentence.remove("<sos>")

    ans = ' '.join(translated_sentence)
    if(len(ans)>1):
      ans = ans[0].upper()+ans[1:]
    if(sentence[-1] in ['.','!','?']):
      ans = ans + sentence[-1]
    return ans

In [ ]:
# Training the model for 20 epochs , by taking batches from the training set iterator made and then
# sending each of the sentences of the batch one by one as input to the encoder and then receiving
# the results from decoder appending it to a set_of_outs array , comparing with the original sentence
# and then backpropagating the loss.
for q in range(0,20):
  print("Epoch Number -> {} / {}".format(q+1, 20))
  for batch in train_iterator:

    hindi_sent = batch.hindi.to(device)
    english_sent = batch.english.to(device)

    # The final set_of_outs vecor which will be having the indexes of the words which will occur with maximum probability 
    set_of_outs = torch.zeros(english_sent.shape[0],hindi_sent.shape[1], len(english.vocab)).to(device)
    
    hidden_state, cell_state = encoder_lstm.forward(hindi_sent)
    
    # SOS TOKEN TO BE FIRST FED TO THE LSTM 
    in_val = english_sent[0]
    
    # The loop is iterated for all the 
    for k in range(1,english_sent.shape[0]):
   
        output, hidden_state, cell_state = decoder_lstm.forward(in_val, hidden_state, cell_state)
        
        set_of_outs[k] = output
        
        #0th dimension is batch size, 1st dimension is word embedding
        #print(the word embedding having the highest probability with respect to the current word and hence
        # get the index corresponding to that word)
        bg2 = output.argmax(1) 
        # Either pass the next word correctly from the dataset or use the earlier predicted word
        in_val = english_sent[k] if random.random() < 0.5 else bg2 
      
    # Clear the accumulating gradients
    e_opt.zero_grad()
    d_opt.zero_grad()
    # Calculate the loss value for every epoch
    # The loss function is such that it does not take into the set_of_outs and english_sent as it is 
    # but we need to convert their dimensionality
    set_of_outs = set_of_outs.view(-1, len(english.vocab))
    # This is done to remove the <sos> token i.e. the first token
    set_of_outs = set_of_outs[1:]
    english_sent= english_sent.view(-1)   
    english_sent= english_sent[1:]
    loss = criterion(set_of_outs,english_sent)
    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()
    # Clip the gradient value is it exceeds > 1 - this is done to avoid the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(encoder_lstm.parameters(), max_norm=1)
    torch.nn.utils.clip_grad_norm_(decoder_lstm.parameters(), max_norm=1)
    # Update the weights values using the gradients we calculated using backpropagation 
    e_opt.step()
    d_opt.step()
    #print("end=======================================================")
  print("Loss in each epoch -> {}".format(loss.item()))
  

In [ ]:
df = pd.read_csv("hindistatements.csv")

df = df["hindi"]


f1=open("answer.txt","a")
for i in range(0,4999):
  src = df.iloc[i]
  res1 = predictions(src, hindi, english, device,largest_sentence=500)
  f1.write(res1)
  f1.write("\n")
 
src = df.iloc[4999]
f1.write(predictions(src, hindi, english, device,largest_sentence=500))

f1.close()

